In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

In [2]:
# kjp attribution:
#   Copied from https://gist.github.com/vgaraujov/fd17b0c151657fbce73189a98617f1c6
#   Everything below this line is copied (including where author derived it from !)


# Code forked from https://www.kaggle.com/jagannathpatta/reading-json-data-getting-dataframe

import pandas as pd
import json

def json_to_dataframe(file):
    f = open ( file , "r") 
    data = json.loads(f.read())               #loading the json file.
    iid = []                                  
    tit = []                                  #Creating empty lists to store values.
    con = []
    Que = []
    Ans_st = []
    Txt = []
    
    for i in range(len(data['data'])):       #Root tag of the json file contains 'title' tag & 'paragraphs' list.
        title = data['data'][i]['title']
        for p in range(len(data['data'][i]['paragraphs'])):  # 'paragraphs' list contains 'context' tag & 'qas' list.
            context = data['data'][i]['paragraphs'][p]['context']
            for q in range(len(data['data'][i]['paragraphs'][p]['qas'])):  # 'qas' list contains 'question', 'Id' tag & 'answers' list.
                question = data['data'][i]['paragraphs'][p]['qas'][q]['question']
                Id = data['data'][i]['paragraphs'][p]['qas'][q]['id']
                for a in range(len(data['data'][i]['paragraphs'][p]['qas'][q]['answers'])): # 'answers' list contains 'ans_start', 'text' tags. 
                    ans_start = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['answer_start']
                    text = data['data'][i]['paragraphs'][p]['qas'][q]['answers'][a]['text']
                    
                    tit.append(title)
                    con.append(context)
                    Que.append(question)                    # Appending values to lists
                    iid.append(Id)
                    Ans_st.append(ans_start)
                    Txt.append(text)

    new_df = pd.DataFrame(columns=['Id','title','context','question','ans_start','text']) # Creating empty DataFrame.
    new_df.Id = iid
    new_df.title = tit           #intializing list values to the DataFrame.
    new_df.context = con
    new_df.question = Que
    new_df.ans_start = Ans_st
    new_df.text = Txt

    final_df = new_df.drop_duplicates(keep='first')  # Dropping duplicate rows from the create Dataframe.
    return final_df

In [3]:
def print_clean(raw):
    print("\n\n".join( raw.split(". ") ) )

In [4]:
# If you have downloaded a SQuAD dataset
# - the following will create a very short sample and save it as a csv tfile

import pathlib
import pandas as pd
import os

source_json = pathlib.Path.home() / os.path.join("Downloads", "train-v2.0.json")
dest_csv =  "squad_show.csv"


squad_json_fname = source_json
squad_df = json_to_dataframe( squad_json_fname)
squad_df.shape

squad_df.iloc[:5].to_csv(dest_csv, index=False)

# From GPT to Bing Search

If you expect a "raw" LLM (e.g., GPT-3) to behave like ChatGPT: you will be disappointed.
- the LLM has been trained to continue the text given in the prompt
- **not** to also be
    - Helpful: answer questions
    - Conversational
    - Harmless


If you want the raw LLM to have capabilities beyond "predict the next" (complete the prompt)
you need to either
- Fine-Tune with examples of the new task
- Condition the text continuation with [specific form of prompts](NLP_Beyond_LLM.ipynb#Pre-train,-prompt,-predict)
    - Exemplars: In-context learning
    - "Pre-prompt" with instructions on desired behavior

ChatGPT is the end-result of several generations of evolution from GPT-3
- using a combination of these techniques

Here is a family tree 

&nbsp; | Model | &nbsp; |
:---|:--:|:---
 | GPT | 
 | $\big\downarrow$ | Fine-tuned to be a helpful Assistant
 | InstructGPT | 
 | $\big\downarrow$ | Fine-tuned to be a Conversational
 | ChatGPT | 
 | $\big\downarrow$ | Non-parametric Knowledge Source
 | WebGPT | 
 | $\big\downarrow$ | Pre-prompt imposes Microsoft standards
 | Bing AI Search | 

We give a very brief overview of some of the key steps on this family tree.

There are a lot of very interesting steps that we omit
- Making GPT helpful, truthful and not harmful

# Fine-tune: Question Answering

ChatGPT is actually based on InstructGPT
- GPT Fine-tuned for question answering

In order to fine-tune a LLM to answer questions
- we can present it with Question/Answer pairs
- formatted as a long text string

    `Question: {question} Answer: {answer}`
- where `{question}` and `{answer}` are place-holders for an example question and its answer.

At inference-time, we just present the question and the request for an Answer

    Question: {question} Answer:
and expect the LLM to complete the text by providing the answer.

SQuAD (Stanford Question Answering Dataset) is a dataset frequently used for Question Answering models.

Each example consists of
- a "context": one or more sentences
- a "question"
- an "answer": a substring of the context

Here are some examples

First: the context (which is shared among the first few questions in this toy dataset).

In [5]:
dest_csv =  "squad_show.csv"
squad_df = pd.read_csv(dest_csv)

example = squad_df.iloc[0]
context = example["context"]

# NOTE: the context doesn't change in the first few examples, so we show it only once
print_clean( example["context"])

Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress

Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child

Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time

Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


And then, some questions and answers

In [6]:
for example_no in range(0,5):
    example = squad_df.iloc[example_no]
    
    question = example["question"]
    text = example["text"]
    
    print("Question:")
    print_clean( question )
    
    print("\n")
    
    answer = example["text"]
    print("Answer:")
    print_clean( answer )
    
    print("\n\n")

Question:
When did Beyonce start becoming popular?


Answer:
in the late 1990s



Question:
What areas did Beyonce compete in when she was growing up?


Answer:
singing and dancing



Question:
When did Beyonce leave Destiny's Child and become a solo singer?


Answer:
2003



Question:
In what city and state did Beyonce  grow up? 


Answer:
Houston, Texas



Question:
In which decade did Beyonce become famous?


Answer:
late 1990s





The answer is a substring of the context, identified by its starting position

In [7]:
print(question)
print("\n")
ans_start = example["ans_start"]
print(f"Answer starts at position {ans_start} ")

print("\nAnswer:\n")
print( context[ ans_start: ans_start+len(answer)])

In which decade did Beyonce become famous?


Answer starts at position 276 

Answer:

late 1990s


# The Pre-prompt

The "pre-prompt" for Bing Search instructs it on how to behave.

The instructions were meant to be hidden, but a simple *adversarial attack* 
caused it to [reveal](https://twitter.com/kliu128/status/1623472922374574080) its pre-prompt.

<img src="images/BingSearch_Sydney_Prepromt.jpeg">

In [8]:
print("Done")

Done
